In [122]:
import pandas as pd
import keras
from keras.models import Sequential,Model
from keras.layers import Dense, Dropout, BatchNormalization,Input
from keras.wrappers.scikit_learn import KerasRegressor
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras import regularizers,initializers
from keras.layers import GaussianNoise
import keras.backend as Kb
import keras.losses

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.utils import class_weight
import numpy as np
from numpy import exp
# Library for Gaussian process
# import GPy
##Library for visualization
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'svg'
import matplotlib;matplotlib.rcParams['figure.figsize'] = (8,6)
import pylab 
import time
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import os
from tqdm import tqdm

In [39]:
# ## Functions for calculation of MSE and MAE
# def mse(y_pred,y_true):
#     mse = np.mean((y_pred-y_true)**2)
#     return mse

# def mae(y_pred,y_true):
#     mae = np.mean(np.absolute(y_pred-y_true))
#     return mae

# s_train, s_test, encoder_train, encoder_test    , y_train, y_test= train_test_split(s, phi_reduce, y, 
#                                                                                 test_size=0.3333)
# N_train = s_train.shape[0]
# N_test = s_test.shape[0]


# # DeepKriging model for continuous data
# model = Sequential()
# # model.add(Dense(100, input_dim = 2,  kernel_initializer='he_uniform', activation='relu'))
# model.add(Dense(100, input_dim = encoder_train.shape[1],  
#             kernel_initializer=initializers.RandomNormal(stddev=0.01), activation='relu'))
# #     model.add(Dense(100, input_dim = encoder_train.shape[1],  kernel_initializer='he_uniform', activation='relu'))
# # model.add(Dropout(rate=0.5))
# # model.add(BatchNormalization())
# #     model.add(Dense(100, activation='relu'))
# model.add(Dense(100, kernel_regularizer=regularizers.L1L2(l1=1e-5, l2=1e-4),
#             bias_regularizer=regularizers.l2(1e-4),
#             activity_regularizer=regularizers.l2(1e-5),activation='relu'))
# #     model.add(Dense(100, activation='relu'))
# model.add(Dense(100, activation='relu'))
# model.add(Dense(100, activation='relu'))
# #     model.add(Dense(100, activation='relu'))
# model.add(Dense(50, activation='relu'))
# # model.add(Dense(100, activation='relu'))
# #model.add(Dropout(rate=0.5))
# model.add(Dense(50, activation='relu'))
# model.add(Dense(50, activation='relu'))
# #model.add(BatchNormalization())
# model.add(Dense(1, activation='linear'))

# # optimizer = keras.optimizers.Adam(lr=0.001)
# # model.compile(optimizer=optimizer, loss='mse', metrics=['mae','mse'])

# # result = model.fit(encoder_train, y_train, 
# #                    validation_data=(encoder_test,y_test), epochs = 100, batch_size = 64, verbose = 2)

# # callbacks = [EarlyStopping(monitor='val_loss', patience=20),
# #              ModelCheckpoint(filepath='comp-2a-1.h5', monitor='val_loss', save_best_only=True)]
# # result = model.fit(encoder_train, y_train, callbacks=callbacks, 
# #                    validation_data=(encoder_test,y_test), epochs = 100, batch_size = 64, verbose = 2)
# # model = keras.models.load_model('comp-2a-1.h5')
# # y_pred = model.predict(encoder_test)


# # # Mean Squared Error
# # mse_var1.append(mse(y_pred[:,0], y_test[:,0]))
# # print(mse_var1)

# # end_time = time.time()
# # print("%s seconds", end_time - start_time)

In [40]:
def deep_model(model, X_train, y_train, X_valid, y_valid, data_type):
    '''
    Function to train a multi-class model. The number of epochs and 
    batch_size are set by the constants at the top of the
    notebook. 
    
    Parameters:
        model : model with the chosen architecture
        X_train : training features
        y_train : training target
        X_valid : validation features
        Y_valid : validation target
    Output:
        model training history
    '''
    NB_START_EPOCHS = 20
    BATCH_SIZE = 512
    if data_type == 'continuous':
        model.compile(optimizer='adam'
                      , loss='mse'
                      , metrics=['mse','mae'])
    if data_type == 'discrete':
        model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    history = model.fit(X_train
                       , y_train
                       , epochs=NB_START_EPOCHS
                       , batch_size=BATCH_SIZE
                       , validation_data=(X_valid, y_valid)
                       , verbose=0)
    return history

def test_model(model, X_train, y_train, X_test, y_test, epoch_stop):
    '''
    Function to test the model on new data after training it
    on the full training data with the optimal number of epochs.
    
    Parameters:
        model : trained model
        X_train : training features
        y_train : training target
        X_test : test features
        y_test : test target
        epochs : optimal number of epochs
    Output:
        test accuracy and test loss
    '''
    BATCH_SIZE = 512
    model.fit(X_train
              , y_train
              , epochs=epoch_stop
              , batch_size=BATCH_SIZE
              , verbose=0)
    results = model.evaluate(X_test, y_test, verbose=0)
    return results
    
def optimal_epoch(model_hist):
    '''
    Function to return the epoch number where the validation loss is
    at its minimum
    
    Parameters:
        model_hist : training history of model
    Output:
        epoch number with minimum validation loss
    '''
    min_epoch = np.argmin(model_hist.history['val_loss']) + 1
    return min_epoch

In [140]:
def mse_func(y_pred,y_true):
    mse = np.mean((y_pred-y_true)**2)
    return mse

def mae_func(y_pred,y_true):
    mae = np.mean(np.absolute(y_pred-y_true))
    return mae
# define and fit the model
def fit_model(X_train, y_train,X_test,y_test):
    # DeepKriging model for continuous data
    ensemble_model = Sequential()

    ensemble_model.add(base_model1)
#     ensemble_model.add(Dense(50, activation = "relu"))
    ensemble_model.add(Dense(50, activation = "relu"))
    ensemble_model.add(Dense(1, activation='linear'))
    ensemble_model.layers[-3].trainable = False
    optimizer = keras.optimizers.Adam(lr=0.01)
    ensemble_model.compile(optimizer=optimizer, loss='mse', metrics=['mse','mae'])
    ensemble_model.fit(X_train, y_train,#validation_data=(X_test,y_test), 
                       epochs = 50, batch_size = 256, verbose = 0)
    return ensemble_model

# fit an ensemble of models
def fit_ensemble(n_members, X_train, Y_train):
    ensemble = list()
    for i in tqdm(range(n_members)):
#         x_train, x_test,y_train,y_test = train_test_split(X_train,Y_train,test_size = 0.1)
        # define and fit the model on the training set
        model = fit_model(X_train, Y_train,X_train,Y_train)
        # evaluate model on the test set
#         yhat = model.predict(x_test, verbose=0)
#         mae1 = mae(yhat, y_test)
#         print('>%d, MAE: %.3f' % (i+1, mae1))
        # store the model
        ensemble.append(model)
    return ensemble
def y_list_uni(yhat,i):
    the_list = list()
    for j in range(len(yhat)):
        the_list.append(yhat[j][0][i])
    return the_list
# make predictions with the ensemble and calculate a prediction interval
def variance(data):
    # Number of observations
    n = len(data)
    # Mean of the data
    mean = sum(data) / n
    # Square deviations
    deviations = [(x - mean) ** 2 for x in data]
    # Variance
    variance = sum(deviations) / (n-1)
    return variance
def predict_with_pi(ensemble, X):
    mean_vec1 = list()
    var_vec1 = list()
    # make predictions
    yhat = [model.predict(X, verbose=0) for model in ensemble]
#     print(yhat)
    for data in tqdm(range(X.shape[0])):
        yhat1 = np.asarray(np.asarray(yhat)[:,data,:])
#         var1_pred = yhat1[:,0]
#         var2_pred = yhat1[:,1]
        mean1 = yhat1.mean()
        var1 = variance(yhat1)
        mean_vec1.append(mean1)
        var_vec1.append(var1)
    return mean_vec1, var_vec1

def calc_distance(s1,s2):
    return(np.sqrt((s1[0] - s2[0])**2 + (s1[1] - s2[1])**2))

def get_nearest_data(s_train,s_test,r,k):
    dist_mat = np.zeros((len(s_test),len(s_train)))
    nearest_var = np.zeros((len(s_test)))
    print(f'running get_nearest_data function ...')
    for i in tqdm(range(len(s_train))):
        for j in range(len(s_test)):
            dist_mat[j][i] = calc_distance(s_train[i],s_test[j])
    
    for i in tqdm(range(len(s_test))):
        a = dist_mat[i]
        b = np.argpartition(a,k)[:k]
        y_val = []
        for index in b:
            y_val.append(r[index])
        nearest_var[i] = (np.mean(y_val))
    print(f'Run finished ...')
    return nearest_var

In [153]:
# num_folds = 5
# kfold = KFold(n_splits=num_folds, shuffle=True, random_state = 123)
# fold_no = 1
# inputs = phi_reduce
# targets = y
# mse_per_fold = []
# mae_per_fold = []
# for train_idx, test_idx in kfold.split(inputs, targets):
#     print('------------------------------------------------------------------------')
#     print(f'Training for fold {fold_no} ...')
#     history = deep_model(model, inputs[train_idx], targets[train_idx]\
#                               , inputs[test_idx], targets[test_idx],'continuous')

#     model_optim = 20#optimal_epoch(history)
#     result = test_model(model, inputs[train_idx], targets[train_idx], inputs[test_idx]\
#                         , targets[test_idx], model_optim)
#     scores = result
#     print(f'The performance of DeepKriging: MSE = {scores[1]}, MAE = {scores[2]}')
#     fold_no = fold_no + 1
#     mse_per_fold.append(scores[1])
# #     mse_per_fold_base.append(scores_base[1])
#     mae_per_fold.append(scores[2])
# #     mae_per_fold_base.append(scores_base[2])

In [179]:
for data_no in range(5):
    df_loc = pd.read_csv("Sub-competition_2a/Training data/LOC_Train_"+str(data_no+1)+".csv",sep = ",", 
                         header = None, names=['x', 'y'])
    df_val = pd.read_csv("Sub-competition_2a/Training data/Z_Train_"+str(data_no+1)+".csv",sep = ",", 
                         header = None, names=['data'])
    df_loc,_,df_val,_ = train_test_split(df_loc, df_val,  
                                                test_size=0.3)
    print('------------------------------------------------------------------------')
    print(f'Training on dataset 2a_{data_no+1}...')
    N = len(df_loc)
    s = np.vstack((df_loc["x"],df_loc["y"])).T
    y = df_val.iloc[:,0].values
    ### Basis functions

    num_basis = [10**2,19**2,37**2]
    knots_1d = [np.linspace(0,1,int(np.sqrt(i))) for i in num_basis]
    ##Wendland kernel
    K = 0
    phi = np.zeros((N, sum(num_basis)))
    for res in range(len(num_basis)):
        theta = 1/np.sqrt(num_basis[res])*2.5
        knots_s1, knots_s2 = np.meshgrid(knots_1d[res],knots_1d[res])
        knots = np.column_stack((knots_s1.flatten(),knots_s2.flatten()))
        for i in range(num_basis[res]):
            d = np.linalg.norm(s-knots[i,:],axis=1)/theta
            for j in range(len(d)):
                if d[j] >= 0 and d[j] <= 1:
                    phi[j,i + K] = (1-d[j])**6 * (35 * d[j]**2 + 18 * d[j] + 3)/3
                else:
                    phi[j,i + K] = 0
        K = K + num_basis[res]
        ## Romove the all-zero columns
    idx_zero = np.array([], dtype=int)
    for i in range(phi.shape[1]):
        if sum(phi[:,i]!=0)==0:
            idx_zero = np.append(idx_zero,int(i))

    phi_reduce = np.delete(phi,idx_zero,1)
    
    num_folds = 2
    kfold = KFold(n_splits=num_folds, shuffle=True, random_state = 123)
    inputs = phi_reduce
    targets = y
    count = 0
    mse_vec = np.empty([num_folds,1])
    mae_vec = np.empty([num_folds,1])
    mpiw_vec = np.empty([num_folds,1])
    picp_vec = np.empty([num_folds,1])
    
    start_time = time.time()
    for train_idx, test_idx in kfold.split(inputs, targets):
        
        
        
        print(f'Cross-validation iteration {count} ...')
        input_dim = Input(shape = (phi_reduce.shape[1], ))
        layer1 = Dense(100, kernel_initializer='he_uniform', activation = 'relu')(input_dim)
        layer2 = Dense(100, activation = 'relu')(layer1)
        layer3 = Dense(100, activation = 'relu')(layer2)
        layer4 = Dense(100, activation = 'relu')(layer3)
        layer5 = Dense(50, activation = 'relu')(layer4)
        layer6 = Dense(50, activation = 'relu')(layer5)
        final_layer = Dense(1, activation = 'linear')(layer5)


        s_train_ensemble, s_train_mse, X_train_ensemble, X_train_mse, \
        y_train_ensemble, y_train_mse = train_test_split(s[train_idx], 
                                                        inputs[train_idx], 
                                                        targets[train_idx], 
                                                test_size=0.25)

        s_train_ensemble1, s_train_ensemble2, X_train_ensemble1, \
        X_train_ensemble2, y_train_ensemble1, y_train_ensemble2 = train_test_split(s_train_ensemble, 
                                                                                  X_train_ensemble, 
                                                                                  y_train_ensemble, 
                                                                                  test_size=0.3)

        base_model = Model(inputs = input_dim, outputs = final_layer)
        optimizer = keras.optimizers.Adam(lr=0.01)
        # Compile the Model
        base_model.compile(optimizer = optimizer, loss = 'mse')
        base_model.fit(X_train_ensemble1, y_train_ensemble1,validation_split = 0.1, epochs = 100,
                        batch_size = 512,verbose = 0)

        base_model1 = Model(inputs = input_dim, outputs = layer5)

        n_members = 10
        ensemble = fit_ensemble(n_members, X_train_ensemble, y_train_ensemble)

        # train data mean and variance vectors
        mean_vec1, var_vec1 = predict_with_pi(ensemble, X_train_mse)

        mean_vec1 = np.asarray(mean_vec1)
        var_vec1 = np.asarray(var_vec1)
        # random error calculation on training data
        r1 = ((y_train_mse - mean_vec1)**2).reshape(y_train_mse.shape[0],1) - var_vec1
        for i in range(len(r1)):
            if r1[i] < 0: r1[i] = 0.0

        # random error calculation on test data with neighbourhood approach

        s_test = s[test_idx]
        y_test = targets[test_idx]
        r1_pred = get_nearest_data(s_train_mse,s_test,r1,50).reshape(s_test.shape[0],1)

        # mean and variance vector for prediction data
        mean_vec1, var_vec1 = predict_with_pi(ensemble, inputs[test_idx])


        end_time = time.time()
#         print("%s seconds", end_time - start_time)
        var_vec1 = np.asarray(var_vec1)
        mean_vec1 = np.asarray(mean_vec1).reshape(var_vec1.shape[0],1)

        # e = np.asarray(e)
        lower_bound1 = mean_vec1 - 1.96*np.sqrt(var_vec1 + r1_pred)
        upper_bound1 = mean_vec1 + 1.96*np.sqrt(var_vec1 + r1_pred)

        count_var0 = 0
        for i in range(len(y_test)):
            if ((y_test[i] > lower_bound1[i]) and (y_test[i] < upper_bound1[i])): count_var0 +=1


        mse = mse_func(mean_vec1,y_test.reshape(var_vec1.shape[0],1))  
        mae = mae_func(mean_vec1,y_test.reshape(var_vec1.shape[0],1))
        picp= count_var0/len(y_test)
        mpiw = np.mean(upper_bound1 - lower_bound1)
        mse_vec[(count-1)] = mse
        mae_vec[(count-1)] = mae
        picp_vec[(count-1)] = picp
        mpiw_vec[(count-1)] = mpiw
        print(f'Performance on cross-validation {count}: MSE = {mse}, MAE = {mae}, picp = {picp}, mpiw = {mpiw}')

    end_time = time.time()
    mse = np.mean(mse_vec)
    mae = np.mean(mae_vec)
    picp = np.mean(picp_vec)
    mpiw = np.mean(mpiw_vec)
    time_t = (start_time - end_time)/num_folds
    print(f'Average performance on 2a-{data_no+1}: MSE = {mse}, MAE = {mae}, picp = {picp}, mpiw = {mpiw}, time = {-time_t}')

# Application on Precipitation data over USA


In [180]:
df = pd.read_csv("precip_data_real.csv",sep = ",")
# df,_= train_test_split(df,
#                                             test_size=0.5)
print('------------------------------------------------------------------------')
print(f'Training on precipitation dataset ...')
N = len(df)
s = np.vstack((df["x"],df["y"])).T
y = df.iloc[:,2].values
### Basis functions

num_basis = [10**2,19**2,37**2]
knots_1d = [np.linspace(0,1,int(np.sqrt(i))) for i in num_basis]
##Wendland kernel
K = 0
phi = np.zeros((N, sum(num_basis)))
for res in range(len(num_basis)):
    theta = 1/np.sqrt(num_basis[res])*2.5
    knots_s1, knots_s2 = np.meshgrid(knots_1d[res],knots_1d[res])
    knots = np.column_stack((knots_s1.flatten(),knots_s2.flatten()))
    for i in range(num_basis[res]):
        d = np.linalg.norm(s-knots[i,:],axis=1)/theta
        for j in range(len(d)):
            if d[j] >= 0 and d[j] <= 1:
                phi[j,i + K] = (1-d[j])**6 * (35 * d[j]**2 + 18 * d[j] + 3)/3
            else:
                phi[j,i + K] = 0
    K = K + num_basis[res]
    ## Romove the all-zero columns
idx_zero = np.array([], dtype=int)
for i in range(phi.shape[1]):
    if sum(phi[:,i]!=0)==0:
        idx_zero = np.append(idx_zero,int(i))

phi_reduce = np.delete(phi,idx_zero,1)

num_folds = 3
kfold = KFold(n_splits=num_folds, shuffle=True, random_state = 123)
inputs = phi_reduce
targets = y
count = 0
mse_vec = np.empty([num_folds,1])
mae_vec = np.empty([num_folds,1])
mpiw_vec = np.empty([num_folds,1])
picp_vec = np.empty([num_folds,1])

start_time = time.time()
for train_idx, test_idx in kfold.split(inputs, targets):
    count += 1
#     df_train = pd.DataFrame(s[train_idx], columns=['lon','lat'])
#     df_train["data"] = y[train_idx]
#     df_test = pd.DataFrame(s[test_idx], columns=['lon','lat'])
#     df_test["data"] = y[test_idx]
#     mypath = "real_data/cross_val_"+str(count)+"/"
#     if not os.path.isdir(mypath):
#         os.makedirs(mypath)
#     df_train.to_csv(mypath+"train.csv",index = False)
#     df_test.to_csv(mypath+"test.csv",index = False)
    print(f'Cross-validation iteration {count} ...')
    input_dim = Input(shape = (phi_reduce.shape[1], ))
    layer1 = Dense(100, kernel_initializer='he_uniform', activation = 'relu')(input_dim)
    layer2 = Dense(100, activation = 'relu')(layer1)
    layer3 = Dense(100, activation = 'relu')(layer2)
    layer4 = Dense(100, activation = 'relu')(layer3)
    layer5 = Dense(50, activation = 'relu')(layer4)
    layer6 = Dense(50, activation = 'relu')(layer5)
    final_layer = Dense(1, activation = 'linear')(layer5)


    s_train_ensemble, s_train_mse, X_train_ensemble, X_train_mse, \
    y_train_ensemble, y_train_mse = train_test_split(s[train_idx], 
                                                    inputs[train_idx], 
                                                    targets[train_idx], 
                                            test_size=0.25)

    s_train_ensemble1, s_train_ensemble2, X_train_ensemble1, \
    X_train_ensemble2, y_train_ensemble1, y_train_ensemble2 = train_test_split(s_train_ensemble, 
                                                                              X_train_ensemble, 
                                                                              y_train_ensemble, 
                                                                              test_size=0.3)

    base_model = Model(inputs = input_dim, outputs = final_layer)
    optimizer = keras.optimizers.Adam(lr=0.01)
    # Compile the Model
    base_model.compile(optimizer = optimizer, loss = 'mse')
    base_model.fit(X_train_ensemble1, y_train_ensemble1,validation_split = 0.1, epochs = 100,
                    batch_size = 512,verbose = 0)

    base_model1 = Model(inputs = input_dim, outputs = layer5)

    n_members = 10
    ensemble = fit_ensemble(n_members, X_train_ensemble, y_train_ensemble)

    # train data mean and variance vectors
    mean_vec1, var_vec1 = predict_with_pi(ensemble, X_train_mse)

    mean_vec1 = np.asarray(mean_vec1)
    var_vec1 = np.asarray(var_vec1)
    # random error calculation on training data
    r1 = ((y_train_mse - mean_vec1)**2).reshape(y_train_mse.shape[0],1) - var_vec1
    for i in range(len(r1)):
        if r1[i] < 0: r1[i] = 0.0

    # random error calculation on test data with neighbourhood approach

    s_test = s[test_idx]
    y_test = targets[test_idx]
    r1_pred = get_nearest_data(s_train_mse,s_test,r1,50).reshape(s_test.shape[0],1)

    # mean and variance vector for prediction data
    mean_vec1, var_vec1 = predict_with_pi(ensemble, inputs[test_idx])


    end_time = time.time()
#         print("%s seconds", end_time - start_time)
    var_vec1 = np.asarray(var_vec1)
    mean_vec1 = np.asarray(mean_vec1).reshape(var_vec1.shape[0],1)

    # e = np.asarray(e)
    lower_bound1 = mean_vec1 - 1.96*np.sqrt(var_vec1 + r1_pred)
    upper_bound1 = mean_vec1 + 1.96*np.sqrt(var_vec1 + r1_pred)

    count_var0 = 0
    for i in range(len(y_test)):
        if ((y_test[i] > lower_bound1[i]) and (y_test[i] < upper_bound1[i])): count_var0 +=1


    mse = mse_func(mean_vec1,y_test.reshape(19918,1))  
    mae = mae_func(mean_vec1,y_test.reshape(19918,1))
    picp= count_var0/len(y_test)
    mpiw = np.mean(upper_bound1 - lower_bound1)
    mse_vec[(count-1)] = mse
    mae_vec[(count-1)] = mae
    picp_vec[(count-1)] = picp
    mpiw_vec[(count-1)] = mpiw
    print(f'Performance on cross-validation {count}: MSE = {mse}, MAE = {mae}, picp = {picp}, mpiw = {mpiw}')

end_time = time.time()
mse = np.mean(mse_vec)
mae = np.mean(mae_vec)
picp = np.mean(picp_vec)
mpiw = np.mean(mpiw_vec)
time_t = (start_time - end_time)/num_folds
print(f'Average performance on precipitation data: MSE = {mse}, MAE = {mae}, picp = {picp}, mpiw = {mpiw}, time = {-time_t}')